

# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.

In [1]:
import random
import string
import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
    """ Converts the generated integers to one-hot vectors """
    ones = torch.sparse.torch.eye(k)
    shape = x.shape
    res = ones.index_select(0, x.view(-1).type(torch.int64))
    return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

    def __init__(self, delay=4, seq_length=15, size=1000):
        self.delay = delay
        self.seq_length = seq_length
        self.size = size

    def __len__(self):
        return self.size

    def __iter__(self):
        """ Iterable dataset doesn't have to implement __getitem__.
            Instead, we only need to implement __iter__ to return
            an iterator (or generator).
        """
        for _ in range(self.size):
            seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
            result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
            yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

def batch_data_generator(single_data_generator, batch_size):
    batch_data = []
    batch_labels = []
    res = []
    for data_point, label in single_data_generator:
        batch_data.append(idx_to_onehot(data_point))
        batch_labels.append(label)
        if len(batch_data) == batch_size:
            res.append((torch.stack(batch_data,dim=0), torch.stack(batch_labels,dim=0)))
            batch_data = []
            batch_labels = []
    # Yield the remaining data if it's less than the batch size
    if batch_data:
        res.append((torch.stack(batch_data,dim=0), torch.stack(batch_labels,dim=0)))
    return res

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`.

In [2]:
def string_to_one_hot(s):
    # Initialize one-hot encoding tensor
    one_hot = torch.zeros(len(s), N+1, dtype=torch.float)
    # Iterate over each character in the string
    for i, char in enumerate(s):
        one_hot[i,ord(char)-96] = 1
    return one_hot

In [3]:
import torch.nn.functional as F

class GRUMemory(torch.nn.Module):

    def __init__(self, hidden_size):
        super().__init__()
        self.gru = torch.nn.GRU(N+1,hidden_size,batch_first=True)
        self.fc1 = torch.nn.Linear(in_features=hidden_size, out_features=128)
        self.fc2 = torch.nn.Linear(in_features=128, out_features=N+1)

    def forward(self, x, hidden=None):
        # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
        # returns:
        # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
        x, hidden = self.gru(x, hidden)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x, hidden

    @torch.no_grad()
    def test_run(self, s):
        # This function accepts one string s containing lowercase characters a-z.
        # You need to map those characters to one-hot encodings,
        # then get the result from your network, and then convert the output
        # back to a string of the same length, with 0 mapped to ' ',
        # and 1-26 mapped to a-z.
        x = string_to_one_hot(s)
        output, _ = self.forward(x)
        output = F.softmax(output,dim=1)
        _ , pred = torch.max(output,dim=1)
        pred = pred.tolist()
        res = ''
        for i in pred:
            if i == 0:
                res += ' '
            else:
                res += chr(i+96)
        return res

## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [4]:
def test_model(model, sequence_length=15):
    """
    This is the test function that runs 100 different strings through your model,
    and checks the error rate.
    """
    total = 0
    correct = 0
    for i in range(500):
        s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
        result = model.test_run(s)
        for c1, c2 in zip(s[:-DELAY], result[DELAY:]):
            correct += int(c1 == c2)
        total += len(s) - DELAY

    return correct / total

In [5]:
batch_size = 64
train_data = batch_data_generator(ds,batch_size)

In [6]:
model = GRUMemory(128)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.005)
n_epochs = 5

In [7]:
def train():
    model.train()
    for epoch in range(1,n_epochs+1):
        correct = 0
        hidden = None
        for batch_idx in range(len(train_data)):
            data, target = train_data[batch_idx]
            if hidden is not None: hidden.detach_()
            logits, hidden = model(data, hidden)
            _ , pred = torch.max(logits,dim=2)
            optimizer.zero_grad()
            #logits = logits.permute(0,2,1)
            loss = criterion(logits.view(-1,N+1), target.view(-1))
            loss.backward()
            optimizer.step()
            correct += (pred == target).int().sum().item()
        print(f'Train Epoch: {epoch}/{n_epochs}, loss: {loss.item():.3f}, accuracy {correct/DATASET_SIZE/15*100:.1f}%')

In [8]:
import time
start_time = time.time()
train()
end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

Train Epoch: 1/5, loss: 0.598, accuracy 66.5%
Train Epoch: 2/5, loss: 0.586, accuracy 72.9%
Train Epoch: 3/5, loss: 0.588, accuracy 73.1%
Train Epoch: 4/5, loss: 0.585, accuracy 73.2%
Train Epoch: 5/5, loss: 0.584, accuracy 73.3%
tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [25]:
import random
import string
import torch
import torch.nn.functional as F

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
    """ Converts the generated integers to one-hot vectors """
    ones = torch.sparse.torch.eye(k)
    shape = x.shape
    res = ones.index_select(0, x.view(-1).type(torch.int64))
    return res.view(*shape, res.shape[-1])


class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

    def __init__(self, max_delay=8, seq_length=20, size=1000):
        self.max_delay = max_delay
        self.seq_length = seq_length
        self.size = size

    def __len__(self):
        return self.size

    def __iter__(self):
        for _ in range(self.size):
            seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
            delay = random.randint(0, self.max_delay)
            result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
            yield seq, delay, result

def batch_data_generator2(k,single_data_generator, batch_size):
    batch_data = []
    batch_labels = []
    batch_delay = []
    res = []
    for data_point, delay, label in single_data_generator:
        batch_data.append(idx_to_onehot(data_point))
        batch_delay.append(delay)
        batch_labels.append(label)
        if len(batch_data) == batch_size:
            res.append((idx_to_onehot(torch.tensor(batch_delay),k+1), torch.stack(batch_data,dim=0), torch.stack(batch_labels,dim=0)))
            batch_data = []
            batch_labels = []
            batch_delay = []
    # Yield the remaining data if it's less than the batch size
    if batch_data:
        res.append((torch.tensor(batch_delay),torch.stack(batch_data,dim=0), torch.stack(batch_labels,dim=0)))
    return res

### Model

And the model.

In [26]:
def string_to_one_hot(s):
    # Initialize one-hot encoding tensor
    one_hot = torch.zeros(len(s), N+1, dtype=torch.float)
    # Iterate over each character in the string
    for i, char in enumerate(s):
        one_hot[i,ord(char)-96] = 1
    return one_hot

In [27]:
class VariableDelayGRUMemory(torch.nn.Module):

    def __init__(self, hidden_size, max_delay):
        super().__init__()
        self.gru = torch.nn.GRU(N+1+9,hidden_size,batch_first=True)
        self.fc1 = torch.nn.Linear(in_features=hidden_size, out_features=128)
        self.fc2 = torch.nn.Linear(in_features=128, out_features=N+1)
        self.max_delay = max_delay

    def forward(self, x, delays):
        # inputs:
        # x - tensor of shape (batch size, seq length, N + 1)
        # delays - tensor of shape (batch size, max_delay+1)
        # returns:
        # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
        replicated_tensor = delays.unsqueeze(1).repeat(1, 20, 1)
        concatenated_tensor = torch.cat((replicated_tensor,x), dim=2)
        x,_ = self.gru(concatenated_tensor)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



    @torch.no_grad()
    def test_run(self, s, delay):
        # This function accepts one string s containing lowercase characters a-z,
        # and a delay - the desired output delay.
        # You need to map those characters to one-hot encodings,
        # then get the result from your network, and then convert the output
        # back to a string of the same length, with 0 mapped to ' ',
        # and 1-26 mapped to a-z.
        x = string_to_one_hot(s).unsqueeze(0)
        delay = idx_to_onehot(torch.tensor([delay]),self.max_delay+1)
        output = self.forward(x, delay)
        output = F.softmax(output,dim=2)
        _ , pred = torch.max(output,dim=2)
        pred = pred.tolist()[0]
        res = ''
        for i in pred:
            if i == 0:
                res += ' '
            else:
                res += chr(i+96)
        return res


### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [28]:
def test_variable_delay_model(model, seq_length=20):
    """
    This is the test function that runs 100 different strings through your model,
    and checks the error rate.
    """
    total = 0
    correct = 0
    for i in range(500):
        s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
        d = random.randint(0, model.max_delay)
        result = model.test_run(s, d)
        if d > 0:
            z = zip(s[:-d], result[d:])
        else:
            z = zip(s, result)
        for c1, c2 in z:
            correct += int(c1 == c2)
        total += len(s) - d

    return correct / total

In [29]:
def train():
    model.train()
    for epoch in range(1,n_epochs+1):
        correct = 0
        for batch_idx in range(len(train_data)):
            delay, data, target = train_data[batch_idx]
            logits = model(data, delay)
            _ , pred = torch.max(logits,dim=2)
            optimizer.zero_grad()
            logits = logits.permute(0,2,1)
            loss = criterion(logits, target)
            loss.backward()
            optimizer.step()
            correct += (pred == target).int().sum().item()
        print(f'Train Epoch: {epoch}/{n_epochs}, loss: {loss.item():.3f}, accuracy {correct/DATASET_SIZE/SEQ_LENGTH*100:.1f}%')

In [14]:
MAX_DELAY = 8
SEQ_LENGTH = 20

DATASET_SIZE = 200000
data_set = VariableDelayEchoDataset(size=DATASET_SIZE)

batch_size = 64
train_data = batch_data_generator2(MAX_DELAY,data_set,batch_size)

In [31]:
import time

start_time = time.time()

# TODO: implement model training here.
model = VariableDelayGRUMemory(128, MAX_DELAY)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.005)
n_epochs = 5

train()

end_time = time.time()
assert end_time - start_time < 600, 'executing took longer than 10 mins'
accuracy = test_variable_delay_model(model)
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

Train Epoch: 1/5, loss: 0.103, accuracy 88.3%
Train Epoch: 2/5, loss: 0.041, accuracy 98.5%
Train Epoch: 3/5, loss: 0.020, accuracy 99.2%
Train Epoch: 4/5, loss: 0.020, accuracy 99.5%
Train Epoch: 5/5, loss: 0.023, accuracy 99.6%
tests passed
